In [42]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from os import listdir
from tqdm import tqdm

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale 
from sklearn.model_selection import StratifiedKFold, ParameterSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

input_path = "../input/"
output_path = "../output/"

In [ ]:
# df = pd.read_csv(output_path+"nn_aug_92303_12_test.csv")
# np.save(output_path+"nn_aug_92303_12_test.npy",df.target)

In [208]:
npys = ([f for f in sorted(listdir(output_path)) if f.endswith("_oof.npy") and f[0]!="!" and "nn" in f 
        and (f.split("_")[2]=='oof.npy' or float(f.split("_")[2]) > 0.921)])
npys

['nn_0.9163_oof.npy',
 'nn_0_0.92162_oof.npy',
 'nn_0_0.92191_oof.npy',
 'nn_0_0.92225_oof.npy',
 'nn_0_0.92226_oof.npy',
 'nn_0_0.92269_oof.npy',
 'nn_0_0.92273_oof.npy',
 'nn_0_0.9229_oof.npy',
 'nn_0_0.92345_oof.npy',
 'nn_1_0.92213_oof.npy',
 'nn_1_0.92217_oof.npy',
 'nn_1_0.92266_oof.npy',
 'nn_1_0.92273_oof.npy',
 'nn_1_0.92294_oof.npy',
 'nn_1_0.92311_oof.npy',
 'nn_1_0.923_oof.npy',
 'nn_2_0.92128_oof.npy',
 'nn_2_0.9228_oof.npy',
 'nn_2_0.92314_oof.npy',
 'nn_2_0.92316_oof.npy',
 'nn_3_0.92114_oof.npy',
 'nn_3_0.92172_oof.npy',
 'nn_3_0.92235_oof.npy',
 'nn_3_0.92269_oof.npy',
 'nn_3_0.92287_oof.npy',
 'nn_3_0.92333_oof.npy',
 'nn_4_0.92263_oof.npy',
 'nn_4_0.9227_oof.npy',
 'nn_4_0.92292_oof.npy',
 'nn_5_0.92241_oof.npy',
 'nn_5_0.92248_oof.npy',
 'nn_5_0.92258_oof.npy',
 'nn_5_0.92286_oof.npy',
 'nn_5_0.92289_oof.npy',
 'nn_aug_0.92303_12_oof.npy']

In [206]:
path = "../kaggle-santander-ctp-2019/predictions/train/"
csvs = [f for f in sorted(listdir(path)) if f.endswith(".csv") and "stacking" not in f][-17:]
csvs

['9197_lightgbm_unique_10x_upsample_0104035943.csv',
 '9199_xgboost_unique_10x_upsample_0104062006.csv',
 '9202_separate_feature_models_threshold_0.84_10fold_04042353.csv',
 '9208_separate_feature_models_threshold_0.80_10fold_07041509.csv',
 '9210_lightgbm_multiplied_counter_10_folds_0604220718.csv',
 '9211_catboost_unbalanced_weights_0304111611.csv',
 '9211_catboost_unique_with_10x_upsample_3103172847.csv',
 '9211_xgboost_multiplied_counter_10_folds_0604015413.csv',
 '9212_catboost_unique_10x_upsample_5_balancer_0204172825.csv',
 '9215_catboost_unique_10x_upsample_3_balancer_0204145755.csv',
 '9218_catboost_excluded_features_10_folds_0804184027.csv',
 '9218_catboost_multiplied_counter_0304011056.csv',
 '9219_catboost_best_parameters_10_folds_0704212230.csv',
 '9222_catboost_multiplied_counter_10_folds_0504034846.csv',
 '9222_catboost_poisson_bootstrap_10_folds_0804042543.csv',
 '9223_catboost_unique_10_fold_20x_upsample_0104001611.csv',
 '9225_catboost_isotonic_09040646.csv']

In [209]:
scaler = StandardScaler() #MinMaxScaler StandardScaler

dt = pd.DataFrame()

df = pd.read_csv(input_path+'train.csv.zip')
df = df[["target"]]

test_filtered = pd.read_pickle(input_path + 'test_filtered.pkl')

cols = []
for file in tqdm(npys):
    train = np.load(output_path+file).reshape(-1, 1)
    col = file[:-4]
    cols.append(col)
    df[col] = scaler.fit_transform(train)
    df[col] = df[col].rank()/len(df)
    
    test = np.load(output_path+file.replace("_oof.npy","_test.npy"))
    test = test[test_filtered.index].reshape(-1, 1)
    
    dt[col] = [x[0] for x in scaler.transform(test)]
    dt[col] = dt[col].rank()/len(dt)
    
for file in tqdm(csvs):
    train = pd.read_csv(path+file)["target"].values.reshape(-1, 1)
    col = file[:-4]
    cols.append(col)
    df[col] = scaler.fit_transform(train)
    df[col] = df[col].rank()/len(df)
    
    test = pd.read_csv((path+file).replace("/train/","/test/"))
    test = test.loc[test_filtered.index]["target"].values.reshape(-1, 1)
    
    dt[col] = [x[0] for x in scaler.transform(test)]
    dt[col] = dt[col].rank()/len(dt)
    
df["avg"] = df[cols].mean(axis=1)
dt["avg"] = dt[cols].mean(axis=1)

dt.tail()

100%|██████████| 17/17 [00:05<00:00,  2.90it/s]


,nn_0.9163_oof,nn_0_0.92162_oof,nn_0_0.92191_oof,nn_0_0.92225_oof,nn_0_0.92226_oof,nn_0_0.92269_oof,nn_0_0.92273_oof,nn_0_0.9229_oof,nn_0_0.92345_oof,nn_1_0.92213_oof,...,9212_catboost_unique_10x_upsample_5_balancer_0204172825,9215_catboost_unique_10x_upsample_3_balancer_0204145755,9218_catboost_excluded_features_10_folds_0804184027,9218_catboost_multiplied_counter_0304011056,9219_catboost_best_parameters_10_folds_0704212230,9222_catboost_multiplied_counter_10_folds_0504034846,9222_catboost_poisson_bootstrap_10_folds_0804042543,9223_catboost_unique_10_fold_20x_upsample_0104001611,9225_catboost_isotonic_09040646,avg
99995,0.95993,0.91587,0.92422,0.92095,0.92141,0.90783,0.91450,0.92132,0.91742,0.91738,...,0.92850,0.93743,0.93774,0.93408,0.92430,0.93139,0.93200,0.92851,0.931530,0.923256
99996,0.14898,0.15590,0.18040,0.18279,0.20509,0.18784,0.15487,0.17513,0.19036,0.19348,...,0.17455,0.16371,0.18725,0.19979,0.17895,0.18717,0.18652,0.15774,0.172650,0.180454
99997,0.62777,0.72123,0.72442,0.72819,0.68325,0.68740,0.71855,0.69727,0.71458,0.68889,...,0.67779,0.70676,0.76420,0.75992,0.78126,0.76486,0.76566,0.73074,0.765670,0.716907
99998,0.43940,0.42042,0.40404,0.38869,0.42218,0.45142,0.38684,0.42630,0.40091,0.43469,...,0.25662,0.29593,0.40004,0.37604,0.38377,0.38189,0.38220,0.35900,0.370095,0.386058
99999,0.25554,0.59828,0.57124,0.57608,0.57963,0.55463,0.59330,0.56882,0.58649,0.55390,...,0.64724,0.65075,0.58023,0.55200,0.57998,0.57700,0.58158,0.68425,0.578365,0.582000


In [ ]:
# df[cols].corr()

In [213]:
test_filtered.index[:10]

Int64Index([3, 7, 11, 15, 16, 17, 18, 20, 21, 22], dtype='int64')

## AVG

In [210]:
roc_auc_score(df.target, df.avg)
# 0.9252533319134989

0.9252354151808034

## LogReg

In [211]:
%%time

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

df["clf"] = 0
dt["clf"] = 0

i = -1
for train_index, valid_index in skf.split(df[cols], df.target):
    i+=1
    
    X_train = df.loc[train_index, cols]
    X_valid = df.loc[valid_index, cols]

    y_train = df.loc[train_index, "target"]
    y_valid = df.loc[valid_index, "target"]
    
    clf = LogisticRegression(C=0.2,
                             solver="newton-cg", 
                             penalty="l2", 
                             n_jobs=-1, 
                             max_iter=100).fit(X_train, y_train) 
    
    y_pred = clf.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "clf"] = y_pred
    print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 5))
    
    dt["clf"] += clf.predict_proba(dt[cols])[:,1] / skf.n_splits
    
print("\nROC AUC:", round(roc_auc_score(df.target, df["clf"]), 5))

# 0 ROC AUC: 0.92511
# 1 ROC AUC: 0.92445
# 2 ROC AUC: 0.92448
# 3 ROC AUC: 0.92458
# 4 ROC AUC: 0.93283
# 5 ROC AUC: 0.92864
# 6 ROC AUC: 0.93127
# 7 ROC AUC: 0.92235
# 8 ROC AUC: 0.92514
# 9 ROC AUC: 0.92289

# ROC AUC: 0.92597
# CPU times: user 10.1 s, sys: 33 s, total: 43.1 s
# Wall time: 2min 39s

# 0 ROC AUC: 0.92516
# 1 ROC AUC: 0.9245
# 2 ROC AUC: 0.92443
# 3 ROC AUC: 0.92452
# 4 ROC AUC: 0.93274
# 5 ROC AUC: 0.92859
# 6 ROC AUC: 0.9312
# 7 ROC AUC: 0.9224
# 8 ROC AUC: 0.92507
# 9 ROC AUC: 0.92297

# ROC AUC: 0.92598
# CPU times: user 10.3 s, sys: 33 s, total: 43.3 s
# Wall time: 2min 36s

0 ROC AUC: 0.92516
1 ROC AUC: 0.92453
2 ROC AUC: 0.92437
3 ROC AUC: 0.92452
4 ROC AUC: 0.93281
5 ROC AUC: 0.92858
6 ROC AUC: 0.93116
7 ROC AUC: 0.92245
8 ROC AUC: 0.9251
9 ROC AUC: 0.92295

ROC AUC: 0.92598
CPU times: user 8.82 s, sys: 31.7 s, total: 40.5 s
Wall time: 3min 8s


In [145]:
# clf_train = df["clf"] 
# clf_test =  dt["clf"] 

In [150]:
df["clf_"] = clf_train 
dt["clf_"] = clf_test 

In [153]:
df["avg"] = (df["clf_"].rank()+df["clf"].rank())/2
dt["avg"] = (dt["clf_"].rank()+dt["clf"].rank())/2

In [154]:
print("\nROC AUC:", round(roc_auc_score(df.target, df["avg"]), 5))


ROC AUC: 0.92598


In [ ]:
# %%time

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# df["clf"] = 0
# dt["clf"] = 0

# i = -1
# for train_index, valid_index in skf.split(df[cols], df.target):
#     i+=1
    
#     X_train = df.loc[train_index, cols]
#     X_valid = df.loc[valid_index, cols]

#     y_train = df.loc[train_index, "target"]
#     y_valid = df.loc[valid_index, "target"]
    
#     clf = LogisticRegression(C=0.5,
#                              solver="newton-cg", 
#                              penalty="l2", 
#                              n_jobs=-1, 
#                              max_iter=100).fit(X_train, y_train) 
    
#     y_pred = clf.predict_proba(X_valid)[:,1] 
#     df.loc[valid_index, "clf"] = y_pred
#     print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 4))
    
#     dt["clf"] += clf.predict_proba(dt[cols])[:,1] / skf.n_splits
    
# print("\nROC AUC:", round(roc_auc_score(df.target, df["clf"]), 5))

## SVM

In [ ]:
# %%time

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# df["svc"] = 0
# dt["svc"] = 0

# i = -1
# for train_index, valid_index in skf.split(df[cols], df.target):
#     i+=1
    
#     X_train = df.loc[train_index, cols]
#     X_valid = df.loc[valid_index, cols]

#     y_train = df.loc[train_index, "target"]
#     y_valid = df.loc[valid_index, "target"]
    
#     svc = SVC(C=10, probability=True).fit(X_train, y_train)  
    
#     y_pred = svc.predict_proba(X_valid)[:,1] 
#     df.loc[valid_index, "svc"] = y_pred
#     print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 4))
    
#     dt["svc"] += svc.predict_proba(dt[cols].values) / skf.n_splits
    
# print("\nROC AUC:", round(roc_auc_score(df.target, df["svc"]), 4))

## NN

In [ ]:
# %%time

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# df["nei"] = 0
# dt["nei"] = 0

# i = -1
# for train_index, valid_index in skf.split(df[cols], df.target):
#     i+=1
    
#     X_train = df.loc[train_index, cols]
#     X_valid = df.loc[valid_index, cols]

#     y_train = df.loc[train_index, "target"]
#     y_valid = df.loc[valid_index, "target"]
    
#     nei = KNeighborsClassifier(n_neighbors=5000, p=1, n_jobs=-1).fit(X_train, y_train) 
    
#     y_pred = nei.predict_proba(X_valid)[:,1] 
#     df.loc[valid_index, "nei"] = y_pred
#     print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 4))
    
#     dt["nei"] += nei.predict_proba(dt[cols])[:,1] / skf.n_splits
    
# print("\nROC AUC:", round(roc_auc_score(df.target, df["nei"]), 4))

In [ ]:
# 0 ROC AUC: 0.9232
# 1 ROC AUC: 0.9232
# 2 ROC AUC: 0.9293
# 3 ROC AUC: 0.9247
# 4 ROC AUC: 0.9216

# ROC AUC: 0.9243
# CPU times: user 7h 3min 54s, sys: 7min 15s, total: 7h 11min 9s
# Wall time: 31min 9s

## LGBM HPO

In [ ]:
rounds = 10000
early_stop_rounds = 500

param_grid = {'num_leaves': list(range(10, 30)),
              'max_bin': [511, 1023, 2047],
              'min_data_in_leaf': list(range(10, 100, 10)),
              'min_sum_hessian_in_leaf': list(range(0, 10)),
              'learning_rate': np.linspace(0.01, 0.03, 31),
              "bagging_freq": list(range(1, 5)),
              "bagging_fraction": np.linspace(0.5, 1.0, 51),
              'feature_fraction': np.linspace(0.08, 1.0, 11),
              'lambda_l1': np.linspace(0, 3, 61),
              'lambda_l2': np.linspace(0, 3, 61)}

param_static = {'boosting_type': 'gbrt',
                'objective': 'binary',
                'tree_learner': 'serial',
                'metric': 'auc',
                'bagging_seed': 42,
                'seed': 42,
                'max_depth': -1,
                'verbose': -1,
                'n_jobs': -1}

param_list = list(ParameterSampler(param_grid, n_iter=100))

rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
                for d in param_list]

results = []
for i, params in enumerate(rounded_list):
    print(params)

    for key in param_static:
        params[key] = param_static[key]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    oof = np.zeros(len(df))
    res = 0

    j = 0
    for train_index, valid_index in skf.split(df[cols], df.target):
        j += 1

        X_train = df.loc[train_index, cols]
        X_valid = df.loc[valid_index, cols]

        y_train = df.loc[train_index, "target"]
        y_valid = df.loc[valid_index, "target"]

        d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
        d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)

        model = lgb.train(params,
                          d_train,
                          num_boost_round=rounds,
                          valid_sets=[d_train, d_valid],
                          valid_names=['train', 'valid'],
                          early_stopping_rounds=early_stop_rounds,
                          verbose_eval=0)

        oof[valid_index] = model.predict(X_valid)
        auc = round(roc_auc_score(y_valid, oof[valid_index]), 4)
        print(i, j, "CV score: {:<8.4f}".format(auc))
    
    res = round(roc_auc_score(df.target, oof), 4)
    print("\noof CV score: {:<8.4f}".format(res))
    
    results.append((params, res))
    for key in param_static:
        del params[key]

    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
    print("best:", sorted_results[0])
    
# 0 ROC AUC: 0.9236
# 1 ROC AUC: 0.9234
# 2 ROC AUC: 0.9299
# 3 ROC AUC: 0.9252
# 4 ROC AUC: 0.9222

# ROC AUC: 0.9246

## LGBM

In [ ]:
def feature_generator(df, vcs_train_test):
    for i in tqdm(range(200)):
        col = "var_" + str(i)
        vtraintest = vcs_train_test[col]
        t = vtraintest[df[col]].fillna(0).values

        df[col + '_train_test_sum_vcs'] = t
        df[col + '_train_test_sum_vcs_prod'] = df[col] * t
        df[col + '_train_test_sum_vcs_unique'] = np.array(t == 1).astype(int)
        df[col + '_train_test_sum_vcs_minus'] = scale(df[col]) - scale(t)
        df[col + '_train_test_sum_vcs_plus'] = scale(df[col]) + scale(t)

In [ ]:
train_df = pd.read_csv(input_path + 'train.csv.zip')

label = train_df.target
train = train_df.drop(['ID_code', 'target'], axis=1)

test = pd.read_csv(input_path + 'test.csv.zip')
test = test.drop(['ID_code'], axis=1)

test_filtered = pd.read_pickle(input_path + 'test_filtered.pkl')
test_filtered = test_filtered.loc[:, train.columns]

train_test = pd.concat([train, test_filtered]).reset_index(drop=True)

vcs_train_test = {}

for col in tqdm(train.columns):
    vcs_train_test[col] = train_test.loc[:, col].value_counts()

feature_generator(train, vcs_train_test)
feature_generator(test, vcs_train_test)

In [182]:
# test_filtered = pd.read_pickle(input_path + 'test_filtered.pkl')
len(test_filtered)

100000

In [ ]:
for col in tqdm(list(train.columns)):
    df[col] = list(train[col])
    dt[col] = list(test[col])

In [ ]:
cols = list(df.columns)
cols.remove("target")
cols.remove("avg")
cols.remove("clf")
cols.remove("blend")
cols.remove("lgb")
len(cols)

In [ ]:
cols

In [ ]:
%%time

rounds = 10000
early_stop_rounds = 300

params = {'num_leaves': 20, 
             'min_sum_hessian_in_leaf': 2, 
             'min_data_in_leaf': 10, 
             'max_bin': 1023, 
             'learning_rate': 0.01, 
             'lambda_l2': 0, 
             'lambda_l1': 1, 
             'feature_fraction': 0.1, 
             'bagging_freq': 2, 
             'bagging_fraction': 0.7,
             'boosting_type': 'gbrt',
             'objective': 'binary',
             'tree_learner': 'serial',
             'metric': 'auc',
             'bagging_seed': 42,
             'seed': 42,
             'max_depth': -1,
             'verbose': -1,
             'n_jobs': -1}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

df["lgb"] = 0
dt["lgb"] = 0

i = -1
for train_index, valid_index in skf.split(df[cols], df.target):
    i+=1
    
    X_train = df.loc[train_index, cols]
    X_valid = df.loc[valid_index, cols]

    y_train = df.loc[train_index, "target"]
    y_valid = df.loc[valid_index, "target"]
    
    d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
    d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)    
    
    model = lgb.train(params,
                      d_train,
                      num_boost_round=rounds,
                      valid_sets=[d_train, d_valid],
                      valid_names=['train','valid'],
                      early_stopping_rounds=early_stop_rounds,
                      verbose_eval=50) 

    y_pred = model.predict(X_valid)
    df.loc[valid_index, "lgb"] = y_pred
    print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 4))
    
    dt["lgb"] += model.predict(dt[cols]) / skf.n_splits
    
print("\nROC AUC:", round(roc_auc_score(df.target, df["lgb"]), 4))

In [ ]:
# 0 ROC AUC: 0.9239
# 1 ROC AUC: 0.9239
# 2 ROC AUC: 0.93
# 3 ROC AUC: 0.9259
# 4 ROC AUC: 0.923

# ROC AUC: 0.9243
# CPU times: user 3h 14min 48s, sys: 49.2 s, total: 3h 15min 37s
# Wall time: 8min 33s

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(15, 100))
lgb.plot_importance(model, max_num_features=len(cols), ax=ax)
plt.title("Light GBM Feature Importance")

In [212]:
w = [0,1]

df["blend"] = (w[1]*df["clf"])/sum(w)
dt["blend"] = (w[1]*dt["clf"])/sum(w)

print("\nROC AUC:", round(roc_auc_score(df.target, df["blend"]), 5))


ROC AUC: 0.92598


In [220]:
sub = pd.read_csv(input_path+'sample_submission.csv.zip')
sub.loc[test_filtered.index,"target"] = dt["blend"]
sub.head(20)

,ID_code,target
0,test_0,0.000000
1,test_1,0.000000
2,test_2,0.000000
3,test_3,0.007340
4,test_4,0.000000
5,test_5,0.000000
6,test_6,0.000000
7,test_7,0.561719
8,test_8,0.000000
9,test_9,0.000000


In [221]:
test_filtered.index

Int64Index([     3,      7,     11,     15,     16,     17,     18,     20,
                21,     22,
            ...
            199980, 199982, 199983, 199984, 199985, 199986, 199993, 199995,
            199996, 199999],
           dtype='int64', length=100000)

In [222]:
sub.to_csv(output_path + "best_blend_non_fake.csv", index=False)

In [175]:
!ls ../output/

 best_auc_nn_0.pkl	  nn_0_0.92226_test.npy   nn_3_0.92114_test.npy
 best_auc_nn_1.pkl	  nn_0_0.92269_oof.npy	  nn_3_0.92172_oof.npy
 best_auc_nn_2.pkl	  nn_0_0.92269_test.npy   nn_3_0.92172_test.npy
 best_auc_nn_3.pkl	  nn_0_0.92273_oof.npy	  nn_3_0.92235_oof.npy
 best_auc_nn.pkl	  nn_0_0.92273_test.npy   nn_3_0.92235_test.npy
 best_blend_10.csv	  nn_0_0.9229_oof.npy	  nn_3_0.92269_oof.npy
 best_blend_13.csv	  nn_0_0.9229_test.npy	  nn_3_0.92269_test.npy
 best_blend_19.csv	  nn_0_0.92345_oof.npy	  nn_3_0.92287_oof.npy
 best_blend_25.csv	  nn_0_0.92345_test.npy   nn_3_0.92287_test.npy
 best_blend_33.csv	  nn_0.9163_oof.npy	  nn_3_0.92333_oof.npy
 best_blend_47.csv	  nn_0.9163_test.npy	  nn_3_0.92333_test.npy
 best_blend_7.csv	  nn_1_0.92213_oof.npy	  nn_4_0.92096_oof.npy
 best_blend_8.csv	  nn_1_0.92213_test.npy   nn_4_0.92096_test.npy
 best_blend_.csv	  nn_1_0.92217_oof.npy	  nn_4_0.92263_oof.npy
 best_blend.csv		  nn_1_0.92217_test.npy   nn_4_0.92263_test.npy
'!cat_0.91558_oof.npy'	  

In [176]:
!kaggle competitions submit -c santander-customer-transaction-prediction -f ../output/best_blend_.csv -m "0.92598 10-folds reg"

100%|██████████████████████████████████████| 6.17M/6.17M [00:05<00:00, 1.22MB/s]
Successfully submitted to Santander Customer Transaction Prediction

In [ ]:
res = 0
ans = {}
for i in range(6):
    with open(f"../output/hpo_logs_{i}.json" ,"r") as f:
        for item in f.readlines():
            d = eval(item)
            ans[d["target"]] = (d["params"],i)
            if d["target"]>=res:
                res = d["target"]
                print(i, res, d["params"])


In [ ]:
for score in sorted(ans)[-4:]:
    print(score, ans[score])

In [ ]:
score = sorted(ans)[-4 - 1]
params = ans[score]
params

In [ ]:
len(ans)